In [1]:
#cosas a importar
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pylab as plt    # import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [2]:
# cargar los datos
df = pd.read_csv('../Data/attacks.csv', encoding='latin1')# encoding='latin1' para corregir un error
pd.set_option('display.max_columns', None)# Muestra todas las columnas
df                    

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Funciones de analisis preliminar de columnas:
def preliminar(df, col):
    print('Type of ', col, ': ', df[col].dtype)
    col = df[col]
    
    if col.dtype == 'object':
        col = col.str.lower().str.rstrip('_').str.lstrip('_')
    
    print('Unique shape is:', col.unique().shape)
    if col.unique().shape[0] < 15:
        print('Unique values are:', col.unique())
    else:
        print('First unique values are:', col.unique()[:5])

    valores_nulos = col.isnull().sum()
    print('NaN values:', valores_nulos)

In [4]:
#guardar las dimensiones originales del dataframe
dim_original = df.shape
df.shape
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
# poner el nombre de todas las columnas en minuscula
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '_').str.rstrip('_')
df.columns


Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number_1', 'case_number_2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')

In [6]:
#para saber el tipo de datos que manejamos
df.info(memory_usage='deep')# vemos la cantidad de valores nulos por columna
# vemos que algunas columnas no tienen el tipo esperable:
error_type = ['case_number', 'date', 'age', 'time', 'case_number1', 'case_number2']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8702 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex                     5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal_(y/n)             5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species                 3464 non-null 

In [7]:
#Valores nulos :
df.info(memory_usage='deep')# vemos la cantidad de valores  no nulos por columna

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8702 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex                     5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal_(y/n)             5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species                 3464 non-null 

In [8]:
#Valores nulos :
valores_nulos = df.isnull().sum().sum()# valores nulos totales en columnas( primer sum()) y en filas(segundo)
valores_nulos # = 488276
nan_cols = df.isna().sum()#miro las columnas con valores nulos #df.isnull().sum()---> Es lo mismo
nan_cols[nan_cols>0] #miro las columnas donde hay valores nulos( si hay valores nulos la suma tiene que ser mayor que 0)
len(nan_cols[nan_cols>0]) # = 24, todas las columnas tienen valores nulos

24

In [9]:
nan_rows = df.isna().sum(axis=1)#miro las filas con valores nulos 
nan_rows[nan_rows > 20] #miro las filas donde hay valores nulos( si hay valores nulos la suma tiene que ser mayor que 0)
len(nan_rows[nan_rows > 20])#  = 19421 filas con valores nulos

19421

In [10]:
#Primera limpia:
df.drop(index=nan_rows[nan_rows > 20].index, inplace=True)#eliminamos todas las filas cuyos valores nulos sean superiores a 20
df.shape , dim_original # Hemos reducido considerablemente el numero de filas al quitar las nulas

((6302, 24), (25723, 24))

In [11]:
#Analizamos las columnas y encontramos relaciones entre ellas:
# las columnas date, case_number, case_number1, case_number2, hablan de fechas(cada una tiene un formato diferente). Se podrian llegar a relacionar con las col 'year' y 'time'
#Las columnas ['country', 'area', 'location'] hablan de localización.
#Las col injury y fatal relacionan el tipo de lesión y si es mortal o no.
#Las col 'href' y 'href_formula' también están relacionadas.
#Las dos ultimas col 'unnamed:_22', 'unnamed:_23' son bastante irrelevantes.

In [12]:
#DEFINICIÖN DEL OBJETIVO: Analizar si los ataques de tiburones se han vuelto más agresivos en los últimos años.
#Para ello redefiniremos el nivel de injury ( leve, grave, muy grave, mortal) usando las col injury y fatal:
#- Categorizar los datos de la col injury( apoyandonos en fatal)
# Utilizaremos las 4 col relacionadas con la fecha para definir nuestro rango temporal:
#-Eliminaremos aquellas filas que no se encuentren en un rango temporal optimo( por no tener suficientes datos dentro del rang)


In [13]:
#ANALISIS DE LA AGRESIVIDAD:

In [14]:
#Analisis preliminar de 'injury'
df['injury'].unique().shape # = 3738 tipos.Demasidos todavía, definir rango temporal antes
df['injury'].unique()
valores_nulos = df['injury'].isnull().sum() # 28 valores nulos
valores_nulos

28

In [15]:
#Analisis preliminar de 'fatal_(y/n)'
df['fatal_(y/n)'].unique().shape # = 9
df['fatal_(y/n)'].unique() #['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y']
valores_nulos = df['fatal_(y/n)'].isnull().sum() # 539 valores nulos
valores_nulos

539

In [16]:
#Limpiar datos de 'fatal_(y/n)':
df['fatal_(y/n)'] = df['fatal_(y/n)'].str.lower().str.rstrip('_').str.lstrip('_')
df['fatal_(y/n)'].unique() #['n', 'y', nan, 'm', 'unknown', '2017'].
fila = df.loc[df['fatal_(y/n)'] == 'm'] #suponemos un error al introducir m = n por cercania en el teclado, Comprobado que la persona no vuelve a aparecer más adelante.
df.loc[285, 'fatal_(y/n)'] = 'n'
fila = df.loc[df['fatal_(y/n)'] == '2017'] #accidente ocurrido en 2012, sin sentido = unknown.
df.loc[786, 'fatal_(y/n)'] = 'unknown'
df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 72. Volveremos a ellos tras categorizar 'injury'.
df['fatal_(y/n)'].fillna('unknown', inplace = True) # rellenamos los nan con 'unknown'.Volveremos a ellos tras categorizar 'injury'.
df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 611.
df['fatal_(y/n)'].unique() # ['n', 'y', 'unknown']
df.shape , dim_original

((6302, 24), (25723, 24))

In [17]:
#Limpiar datos de 'injury':
df['injury'] = df['injury'].str.lower().str.rstrip('_').str.lstrip('_')
df.loc[df['fatal_(y/n)'] == 'y', 'injury'] = 'fatal' # para cada valor que sea y(yes) en 'fatal' queremos que injury = 'fatal'
df['injury'].unique().shape # = 3222 tipos diferentes todavía
filt = df['injury'].str.contains('fatal', na=False) # comprobamos que valores contienen 'fatal', 
injury_fatal = df.loc[filt, 'injury']
injury_fatal # = 1447 con fatal en algún punto
filt1 = df['injury'].str.contains('fatal') & df['injury'].str.contains('questionable')
print(df['injury'][5590]) # hay un dato cuestionable -> 5590. No se sabe la especie.Eliminamos fila porque no se sabe si es o no un tiburon
df.drop(index = 5590)
df.loc[df['injury'] == 'fatal'].shape # = 1409 con sólo fatal.Vamos a rellenar todos los que tengan fatal con fatal directamente
df['injury'] = df['injury'].str.replace('.*fatal.*', 'fatal', regex = True)
df.loc[df['injury'] == 'fatal'].shape # = 1447 con 'fatal'.Limpio.
df['injury'].isnull().sum() # = 27. Uno menos que antes
df['injury'].fillna('unknown', inplace = True)
df.loc[df['injury'] == 'unknown'].shape # = 29
df['injury'].isnull().sum() # = 0 . No sabemos que tipo de lesión se produjo. Asi que vamos a eliminar las filas porque no son relevantes para nuestro objeto de estudio
df.drop(df.loc[df['injury'] == 'unknown'].index, inplace = True)
df['injury'].unique().shape # = 3189 tipos difetentes. Complicado aún, vamos a tratar de eliminar las filas que no sepamos si son ataques de tiburón y  que estén fuera de nuestro rango temporal

df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 531
df.loc[df['injury'] == 'fatal', 'fatal_(y/n)'] = 'y' #limpiamos col 'fatal' con 'injury'
df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 531 no ha aumentado
df.shape , dim_original

reported fatal, but this is a questionable incident


((6273, 24), (25723, 24))

In [18]:
#Analisis preliminar de 'species':
df['species'].unique().shape                 # = 1547 tipos
df['species'].unique()
valores_nulos = df['species'].isnull().sum() # 2821 valores nulos
valores_nulos

2821

In [19]:
#Limpiar datos de 'species':
# queremos buscar datos en los que no estemos seguros que sea un tiburon. La especie no nos importa
df['species'] = df['species'].str.lower().str.rstrip('_').str.lstrip('_')
df['species'].fillna('unknown', inplace = True)
df['species'].unique().shape # = 1532 tipos
filtro = ~df['species'].str.contains(r'shark|unknown', regex=True)
valores = df[filtro]['species']
valores.shape # = 234 valores que no tienen shark
valores.unique().shape# = 49 valores unicos que no tienen shark, vamos a limpiarlos: si es questionable, invalido, eliminamos fila si incluye los char shar lo cambiamos por shark
valores.unique()
df = df[~df['species'].str.contains('questionable|invalid', case=False, na=False, regex=True)]
df['species'].unique().shape # = 1520 tipos
valores.unique().shape # = 45 valores unicos que no tienen shark
valores.shape # = 65 valores
df['species'] = df['species'].str.replace('.*shar|wobbegong|porbeagle|bronze whaler|hammerhead|spurdog|dogfish|blue pointer|c. .*', 'shark', regex=True) # Sustituimos todos los valores de la columna que contengan *shar* en algún punto por 'shark'
valores.unique().shape # = 19 valores unicos que no tienen shark
valores.shape # = 34 valores en total
valores
new_df = df.loc[df['species'].isin(valores)] # almacenamos los valores que nos quedan en un nuevo dt
new_df.shape# comprobamos esos 34 valores, en los cuales no queda claro si es o no un tiburon
df.drop(new_df.index, inplace = True)#al no estar seguros si se trata o no de un tiburon, eliminamos las filas
df.loc[df['species']== 'unknown'].shape # Recordar que tenemos 2822 filas que no estamos seguros de que se trate de un tiburon
df.shape , dim_original

((6059, 24), (25723, 24))

In [20]:
#Analisis de las columna 'year':
preliminar(df, 'year')


Type of  year :  float64
Unique shape is: (242,)
First unique values are: [2018. 2017.   nan 2016. 2015.]
NaN values: 2


In [34]:
#tenemos dos valores nulos vamos a buscarlos
indices_nulos = df[df['year'].isnull()].index
indices_nulos # = 187 y 6079
df.loc[187] # es 2017 lo sustituimos
df.loc[6079] # es 1836 lo sustituimos
df['year'][187] = 2017
df['year'][6079] = 1836
preliminar(df, 'year')

Type of  year :  float64
Unique shape is: (241,)
First unique values are: [2018. 2017. 2016. 2015. 2014.]
NaN values: 0


In [44]:
df['year'] = df['year'].astype(str) #convertivos la columna a string
def limpiar_año(valor):
    patron = r"\d{4}"
    coincidencias = re.findall(patron, valor)
    if coincidencias:
        return coincidencias[0]
    else:
        return valor
df['year'] = df['year'].apply(limpiar_valor)
preliminar(df, 'year')



Type of  year :  object
Unique shape is: (241,)
First unique values are: ['2018.0' '2017.0' '2016.0' '2015.0' '2014.0']
NaN values: 0


6151    1751.0
6152    1749.0
6154    1748.0
6155    1742.0
6156    1738.0
6157    1733.0
6158    1723.0
6159    1721.0
6160    1703.0
6161    1700.0
6162    1700.0
6163    1700.0
6168    1642.0
6169    1595.0
6170    1580.0
6173    1543.0
6174     500.0
6175      77.0
6176       5.0
6177       0.0
Name: year, dtype: object

In [23]:
#Analisis de las columnas relacionadas con la fecha:
df.head()

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"no injury to occupant, outrigger canoe and pad...",n,18h00,sharkk,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,minor injury to left thigh,n,14h00 -15h00,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,injury to left lower leg from surfboard skeg,n,07h45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,minor injury to lower leg,n,NaN,sharkk,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,lacerations to leg & hand shark provoked incident,n,NaN,"sharkk, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [24]:
#Analisis preliminar de 'case_number'
preliminar(df, 'case_number')

Type of  case_number :  object
Unique shape is: (6044,)
First unique values are: ['2018.06.25' '2018.06.18' '2018.06.09' '2018.06.08' '2018.06.04']
NaN values: 1


In [25]:
#Analisis preliminar de 'date'
preliminar(df, 'date')

Type of  date :  object
Unique shape is: (5239,)
First unique values are: ['25-jun-2018' '18-jun-2018' '09-jun-2018' '08-jun-2018' '04-jun-2018']
NaN values: 0


In [26]:
#Analisis preliminar de 'case_number_1'
preliminar(df, 'case_number_1')

Type of  case_number_1 :  object
Unique shape is: (6043,)
First unique values are: ['2018.06.25' '2018.06.18' '2018.06.09' '2018.06.08' '2018.06.04']
NaN values: 0


In [27]:
#Analisis preliminar de 'case_number_2'
preliminar(df, 'case_number_2')

Type of  case_number_2 :  object
Unique shape is: (6044,)
First unique values are: ['2018.06.25' '2018.06.18' '2018.06.09' '2018.06.08' '2018.06.04']
NaN values: 0


In [28]:
# Vamos a convertir todos los valores de cada columna al mismo formato: year_month_day
#case_number: tiene un valor nulo, vamos a corregirlo

columna = 'case_number'
indices_nulos = df[df[columna].isnull()].index.tolist()
indices_nulos # el indice erróneo es 5488
df.loc[5488] #Lo buscamos y vemos que podemos completarlo con una de las otras columnas, en este caso case_number_1
df['case_number'][5488] = df['case_number_1'][5488]




In [29]:

def limpiar_valor(valor):
    patron = r"\d{4}\.\d{2}\.\d{2}"
    coincidencias = re.findall(patron, valor)
    if coincidencias:
        return coincidencias[0]
    else:
        return valor
#definimos una funcion que elimine las letras de los valores que sigan el patron dado
df['case_number'] = df['case_number'].apply(limpiar_valor)
cantidad_no_cumplen_patron = len(df[~df['case_number'].str.contains(r"\d{4}\.\d{2}\.\d{2}")])
cantidad_no_cumplen_patron # comprobamos cuantos valores no siguen el patron 

124

In [30]:
#df = df[df['columna'].str.match(r"\d{4}\.\d{2}\.\d{2}")]